DN: Take Home Assignment
Date Range: 2020-2023

Ingestion
- Ingest Data
  - XML,CSV-bar,JSON
- Create Ticket Fee refence te
eEDAable
- Identify Police Officers in data
- Identify Officer with most speeding tickets issued
- What 3 months had the most speeding tickets?
- Are there any trends
 - Monthly
 - Yearly
-Top 10 people who have spent the most on speeding tickets


In [113]:
# !pip install spark-xml
# !pip install delta-spark

from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

ERROR: Could not find a version that satisfies the requirement spark-xml (from versions: none)
ERROR: No matching distribution found for spark-xml


In [192]:
# XML is not included in the Python release it seems, only Scala
# https://github.com/databricks/spark-xml
from pyspark.sql.column import Column, _to_java_column
from pyspark.sql.types import StructType
from pyspark.sql.utils import *

def ext_from_xml(xml_column, schema, options={}):
    java_column = _to_java_column(xml_column.cast('string'))
    java_schema = spark._jsparkSession.parseDataType(schema.json())
    scala_map = spark._jvm.org.apache.spark.api.python.PythonUtils.toScalaMap(options)
    jc = spark._jvm.com.databricks.spark.xml.functions.from_xml(
        java_column, java_schema, scala_map)
    return Column(jc)

def ext_schema_of_xml_df(df, options={}):
    assert len(df.columns) == 1

    # scala_options = spark._jvm.PythonUtils.toScalaMap(options)
    scala_options = {}
    
    java_xml_module = getattr(getattr(
        spark._jvm.com.databricks.spark.xml, "package$"), "MODULE$")
    java_schema = java_xml_module.schema_of_xml_df(df._jdf, scala_options)
    return _parse_datatype_json_string(java_schema.json())

In [193]:
ticket_fees = [{'type':'base', 'value':30},
               {'type':'base + school zone','value': 60},
               {'type':'base + construction work zone','value':60},
               {'type':'base + school zone + construction work zone', 'value':120}]

keywords = ['automobiles', 'people', 'speeding_tickets']

path = './work/ttpd_data'
all_files = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
# all_files

In [196]:
def _get_spark_session_() -> SparkSession:
    builder = SparkSession \
        .builder \
        .appName('takehome') \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.16.0") 

    return configure_spark_with_delta_pip(builder).getOrCreate()


def _read_file_(spark, file_path):
    
    if file_path.endswith('.xml'):
        print(f'Reading XML file {file_path}')
        xml = spark.read.text(file_path)
        
        # https://medium.com/@pawankumarshukla_57258/spark-structured-streaming-read-and-process-data-xml-and-json-from-azure-event-hub-4030567cff44
        df_schema = spark.read.format("binaryFile").load(file_path).selectExpr("CAST(content AS STRING)")
        xml_extraction_options = {"mode": "DROPMALFORMED","inferSchema":"true"}
        
        xml_schema = ext_schema_of_xml_df(df_schema.select("content"))
        
        # df=df.select(ext_from_xml(col("body"), xmlschema, xml_extraction_options).alias("xmldf"))
        # schema = ext_schema_of_xml_df(xml.select("value"))
        df_schema.show()

        df =[]
        return df

    elif file_path.endswith('.json'):
        # print(f'Reading JSON file {file_path}')
        df = spark.read.json(file_path)
        if 'speeding_tickets' in df.columns:
            df = df.select(explode(col('speeding_tickets')).alias('ticket')).select('ticket.*')
        return df
        
    elif file_path.endswith('.csv'):
        # print(f'Reading CSV file {file_path}')
        return spark.read.csv(file_path, header=True, inferSchema=True,sep='|')
    else:
        return None


def _combine_files_(spark, keyword):
    filtered_files = [f for f in all_files if keyword in f]
    if not filtered_files:
        return None

    dataframes = [_read_file_(spark, f) for f in filtered_files if _read_file_(spark, f) is not None]
    if not dataframes:
        return None

    combined_df = dataframes[0]
    for df in dataframes[1:]:
        combined_df = combined_df.union(df)
    return combined_df


def main():
    spark = _get_spark_session_()
    print(f'Spark Session Initialized...')

    combined_dfs = {keyword: _combine_files_(spark, keyword) for keyword in keywords}

    for keyword, df in combined_dfs.items():
        if df is not None:
            print(f"Combined DataFrame for {keyword}:")
            df.show()
        else:
            print(f"No files found for keyword: {keyword}")
    fee_df = spark.createDataFrame(ticket_fees)
    fee_df.show()

In [ ]:
# main()

spark = _get_spark_session_()
_read_file_(spark, './work/ttpd_data/20240503111609_automobiles_41.xml')